# Vigilant Asset Allocation (VAA)

Vigilant Asset Allocation (VAA) is introduced by Wouter J. Keller and Jan Willem Keuning in research paper "Breadth Momentum and Vigilant Asset Allocation (VAA): Winning More by Losing Less" (https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3002624)


In [2]:
import pandas as pd
import src.fmp as fmp

## Portfolio Assets

In [3]:
offensive = ['SPY', 'VEA', 'VWO', 'AGG']
defensive = ['SHY', 'IEF', 'LQD']

## Upgrade version of dual momentum

1. We buy the strongest asset in terms of momentum in offensive assets.
2. If the most recent return of any of the offensive assets is negative, we go to defensive asset.

## How to compute recent returns and weights

recent_return = 12 * (recent 1M return) + 4 * (recent 4M return) + 2 * (recent 6M return) + 1 * (recent 12M return)

In [7]:
offensive_prices = pd.DataFrame()

for symbol in offensive:
    data = fmp.get_daily_prices(symbol)
    offensive_prices[symbol] = data['Close']

offensive_prices

,SPY,VEA,VWO,AGG
Date,,,,
2016-09-23,215.990005,37.439999,37.689999,112.230003
2016-09-26,214.240005,37.110001,37.230000,112.370003
2016-09-27,215.570007,37.290001,37.619999,112.510002
2016-09-28,216.639999,37.619999,37.990002,112.540001
2016-09-29,214.679993,37.160000,37.360001,112.540001
...,...,...,...,...
2021-09-16,447.170000,52.930000,51.650000,115.970000
2021-09-17,441.400000,52.270000,51.450000,115.870000
2021-09-20,434.040000,51.080000,49.590000,116.090000


## Offensive Assets Momentum

In [8]:
mom = {'1M': [], '3M': [], '6M': [], '12M': []}

for symbol in offensive:
    mom['1M'].append(fmp.calculate_hist_momentum(symbol, 30))
    mom['3M'].append(fmp.calculate_hist_momentum(symbol, 90))
    mom['6M'].append(fmp.calculate_hist_momentum(symbol, 180))
    mom['12M'].append(fmp.calculate_hist_momentum(symbol, 252))

mom

{'1M': [-0.013339943215106493,
  -0.016891250711710014,
  -0.02160910669496426,
  0.005539207200969366],
 '3M': [0.05376398630969352,
  0.007194244604316497,
  -0.02160910669496426,
  0.017338030227794232],
 '6M': [0.1721590525121367,
  0.06716108225873338,
  -0.006076048608388911,
  -0.009885768191094811],
 '12M': [0.3571162275082339,
  0.2850409008920639,
  0.19938508040172861,
  -0.016090802572987648]}

In [13]:
offensive_momentum = pd.DataFrame(mom, index=offensive)
offensive_momentum

,1M,3M,6M,12M
SPY,-0.013340,0.053764,0.172159,0.357116
VEA,-0.016891,0.007194,0.067161,0.285041
VWO,-0.021609,-0.021609,-0.006076,0.199385
AGG,0.005539,0.017338,-0.009886,-0.016091


In [15]:
offensive_momentum[
    'Score'] = 12 * offensive_momentum['1M'] + 4 * offensive_momentum[
        '3M'] + 2 * offensive_momentum['6M'] + 1 * offensive_momentum['12M']
offensive_momentum

,1M,3M,6M,12M,Score
SPY,-0.013340,0.053764,0.172159,0.357116,0.756411
VEA,-0.016891,0.007194,0.067161,0.285041,0.245445
VWO,-0.021609,-0.021609,-0.006076,0.199385,-0.158513
AGG,0.005539,0.017338,-0.009886,-0.016091,0.099960


## Defensive Assets Momentum

In [16]:
mom = {'1M': [], '3M': [], '6M': [], '12M': []}

for symbol in defensive:
    mom['1M'].append(fmp.calculate_hist_momentum(symbol, 30))
    mom['3M'].append(fmp.calculate_hist_momentum(symbol, 90))
    mom['6M'].append(fmp.calculate_hist_momentum(symbol, 180))
    mom['12M'].append(fmp.calculate_hist_momentum(symbol, 252))

mom

{'1M': [-0.000696136442742804, -0.004375053615853058, 0.007676255775823529],
 '3M': [-0.0018542241064524762, 0.017713074204550372, 0.036330190848470986],
 '6M': [-0.0026632815810180626, -0.021251492304747904, -0.00478434414261489],
 '12M': [-0.0041623078774958985, -0.048532569719644904, 0.006326280095815099]}

In [17]:
defensive_momentum = pd.DataFrame(mom, index=defensive)
defensive_momentum

,1M,3M,6M,12M
SHY,-0.000696,-0.001854,-0.002663,-0.004162
IEF,-0.004375,0.017713,-0.021251,-0.048533
LQD,0.007676,0.036330,-0.004784,0.006326


In [18]:
defensive_momentum['Score'] = 12 * defensive_momentum['1M'] + 4 * defensive_momentum['3M'] + 2 * defensive_momentum['6M'] + 1 * defensive_momentum['12M']
defensive_momentum

,1M,3M,6M,12M,Score
SHY,-0.000696,-0.001854,-0.002663,-0.004162,-0.025259
IEF,-0.004375,0.017713,-0.021251,-0.048533,-0.072684
LQD,0.007676,0.036330,-0.004784,0.006326,0.234193
